In [1]:
import os
from math import *
from time import perf_counter_ns
import gc
from random import choice
os.chdir('tsp_dataset')

# Parse input 

In [2]:
def parse(fileaddress):
    CoOrdinates = {}
    with open(fileaddress) as f:
        lines = f.readlines()
    lines[0] = ': '.join(lines[0].split(' : '))
    Name = lines[0].split(" ")[1].strip()
    lines[4] = ': '.join(lines[4].split(' : '))
    EWT = lines[4].split(" ")[1].strip()
    lines[3] = ': '.join(lines[3].split(' : '))
    Dimension = lines[3].split(" ")[1].strip()
    flag = False
    for line in lines:
        line = ' '.join(line.split())
        if line.strip().split(" ")[0] == '1':
            flag = True
        elif flag == False:
            continue
        if line.strip().split(" ")[0] == 'EOF':
            break
        n,x,y = line.strip().split(" ")
        CoOrdinates[n] = (x,y)
    return Name, EWT, Dimension, CoOrdinates

<p>we save the coordinates in a dictionary with the vertex ID as the key and a (x,y) tuple as the data</p>

In [3]:
NAME , EDGE_WEIGHT_TYPE , DIMENSION , Coordinates  = parse('berlin52.tsp')
print(f"Data set name: {NAME}\nType: {EDGE_WEIGHT_TYPE}\nDimension: {DIMENSION}\nCoordinates: {Coordinates}")

Data set name: berlin52
Type: EUC_2D
Dimension: 52
Coordinates: {'1': ('565.0', '575.0'), '2': ('25.0', '185.0'), '3': ('345.0', '750.0'), '4': ('945.0', '685.0'), '5': ('845.0', '655.0'), '6': ('880.0', '660.0'), '7': ('25.0', '230.0'), '8': ('525.0', '1000.0'), '9': ('580.0', '1175.0'), '10': ('650.0', '1130.0'), '11': ('1605.0', '620.0'), '12': ('1220.0', '580.0'), '13': ('1465.0', '200.0'), '14': ('1530.0', '5.0'), '15': ('845.0', '680.0'), '16': ('725.0', '370.0'), '17': ('145.0', '665.0'), '18': ('415.0', '635.0'), '19': ('510.0', '875.0'), '20': ('560.0', '365.0'), '21': ('300.0', '465.0'), '22': ('520.0', '585.0'), '23': ('480.0', '415.0'), '24': ('835.0', '625.0'), '25': ('975.0', '580.0'), '26': ('1215.0', '245.0'), '27': ('1320.0', '315.0'), '28': ('1250.0', '400.0'), '29': ('660.0', '180.0'), '30': ('410.0', '250.0'), '31': ('420.0', '555.0'), '32': ('575.0', '665.0'), '33': ('1150.0', '1160.0'), '34': ('700.0', '580.0'), '35': ('685.0', '595.0'), '36': ('685.0', '610.0'), 

In [4]:
def computeWeight(First, Second, Type):
    x1 = float(First[0])
    x2 = float(Second[0])
    y1 = float(First[1])
    y2 = float(Second[1])
    if Type == "EUC_2D":
        return sqrt((x1-x2)**2 + (y1-y2)**2)
    if Type == "GEO":
        PI = 3.141592
        R = 6378.388 
        deg = int(x1)
        minn = x1 - deg
        lon1 = PI * (deg + 5.0 * minn/ 3.0) / 180.0
        deg = int(x2)
        minn = x2 - deg
        lon2 = PI * (deg + 5.0 * minn/ 3.0) / 180.0
        deg = int(y1)
        minn = y1 - deg
        lat1 = PI * (deg + 5.0 * minn/ 3.0) / 180.0
        deg = int(y2)
        minn = y2 - deg
        lat2 = PI * (deg + 5.0 * minn/ 3.0) / 180.0
        dlon = cos(lon2 - lon1)
        dlat = cos(lat2 - lat1)
        slat = cos(lat2 + lat1)
        di = R * acos( 0.5*((1.0+dlon)*dlat - (1.0-dlon)*slat) ) + 1.0
        return di

In [5]:
print(Coordinates['1'],Coordinates['2'])
print(computeWeight(Coordinates['1'],Coordinates['2'],EDGE_WEIGHT_TYPE))

('565.0', '575.0') ('25.0', '185.0')
666.1080993352356


In [6]:
def getKey(sdict):
    key_list = list(sdict.keys())
    return key_list

def makeGraph(Coordinates):
    keys = getKey(Coordinates)
    matrix = []
    for i in range(len(keys)):
        u = keys[i]
        for j in range(i,len(keys)):
            v = keys[j]
            if v == u:
                continue
            w = computeWeight(Coordinates[u],Coordinates[v],EDGE_WEIGHT_TYPE) # weight computed for edge(u,v)
            matrix.append((u,v,w))
    return keys, matrix

def makeDict(Coordinates):
    keys = getKey(Coordinates)
    dictionary = {}
    for i in range(len(keys)):
        u = keys[i]
        for j in range(i,len(keys)):
            v = keys[j]
            if v == u:
                continue
            w = computeWeight(Coordinates[u],Coordinates[v],EDGE_WEIGHT_TYPE) # weight computed for edge(u,v)
            dictionary[(u,v)] = (u,v,w)
    return dictionary

### Nearest Neighbor Algorithm

In [7]:
"""
Nearest Neighbor Algorithm
sort the edges
pick the first node
"""
#  {"1": [(2,w), (3,w), ...]...}
# creating adjacency matrix with sorted weights
def make_set_nn(all_edges):
    nn_weights = {}
    for e in all_edges:
        if e[0] not in nn_weights:
            nn_weights[e[0]] = []
        # add edges to the adjacency list    
        if e[1] not in nn_weights:
            nn_weights[e[1]] = []
        nn_weights[e[0]].append((e[1], e[2]))
        nn_weights[e[1]].append((e[0], e[2]))
    # sort weights
    for vertice in nn_weights:
        nn_weights[vertice] = sorted(nn_weights[vertice], key=lambda el: el[1])
        
    return nn_weights



def nearest_neighbour(vertice, matrix_nn, visited, path):
    # if we saw the vertex before, return
    if vertice not in matrix_nn or len(matrix_nn[vertice]) == 0:
        return 
    
    # get the lightest edge
    lightest = matrix_nn[vertice].pop(0)    
    # if we saw the neighbour before, go forward
    while lightest[0] in visited and len(matrix_nn[vertice]) > 0:
        lightest = matrix_nn[vertice].pop(0)
    
    # found the unseen one, remove fron adj_list
    matrix_nn.pop(vertice)
    # append the edge to a path
    path.append((vertice , lightest[0], lightest[1]))
    # populate the visited set
    visited.add(lightest[0])
    # call recursively for another vertice
    nearest_neighbour(lightest[0], nn_mtrx, visited, path)

def nnWeight(solution):
    return sum([edge[2] for edge in solution])

In [8]:
nnSol = []
for file in os.listdir():
    NAME , EDGE_WEIGHT_TYPE , DIMENSION , Coordinates  = parse(file)
    vertices, edges = makeGraph(Coordinates)
    # define source
    source = vertices[0]
    # create adjacency matrix
    nn_mtrx = make_set_nn(edges)
    gc.disable()
    start_time = perf_counter_ns()
    path = []
    # save the adjacency for the source vertex
    source_list = nn_mtrx[source]
    visited = set([source])
    nearest_neighbour(source, nn_mtrx, visited, path)
    # create a cycle
    last_edge = [s for s in source_list if s[0] == path[-1][0]][0]
    cycle = path + [(last_edge[0], source, last_edge[1])]
    end_time = perf_counter_ns()
    time = end_time - start_time
    gc.enable()
    weight = nnWeight(cycle)
    print("did",NAME,"in",time,"ns with a weight of:",weight)
    nnSol.append((NAME,weight,time))

did berlin52 in 175200 ns with a weight of: 10696.967521046923
did burma14 in 87900 ns with a weight of: 2102.247219722056
did ch150 in 807900 ns with a weight of: 8971.96390462492
did d493 in 18509900 ns with a weight of: 47598.04750184437
did dsj1000 in 123200000 ns with a weight of: 26002494.412531007
did eil51 in 7903100 ns with a weight of: 586.5277687812595
did gr202 in 2567300 ns with a weight of: 73440.8841090444
did gr229 in 3665600 ns with a weight of: 176314.83208294824
did kroA100 in 915800 ns with a weight of: 31006.17051322257
did kroD100 in 867000 ns with a weight of: 30653.183734586575
did pcb442 in 13547700 ns with a weight of: 66825.5346316779
did ulysses16.tsp in 212000 ns with a weight of: 13286.858017454517
did ulysses22.tsp in 51600 ns with a weight of: 13417.393044107894


### Efficient Kruskal 

In [9]:
# Graph object
class Graph:
    def __init__(self, V, E, num_V, num_E):
        self.V = V
        self.E = E
        self.num_V = num_V
        self.num_E = num_E

In [71]:
"""
Kruskal Efficient Agorithm
sort the edges
make a list of vertices
get the first edge
find the parent of the vertices in the edge(u,v)
if there is no loop union
"""
class Kruskal_Efficient:
    
    def __init__(self, graph):
        self.graph = graph
        self.sets = {} # set of vertices
        self.MST = [] # Minimum Spanning Tree
    
    
    # make a set of vertices
    def make_sets(self):
        for v in self.graph.V:
            self.sets[v] = [v]
        
    
    
    # union the subsets which the vertices are not in the same sets
    def union(self, u_prnt, v_prnt):
        # get the size of two elements and append the vertices to the bigger one
        if (len(self.sets.get(u_prnt)) >= len(self.sets.get(v_prnt))):
            self.sets[u_prnt].extend(self.sets[v_prnt])
            self.sets.pop(v_prnt)
        
        # append the list of vertices of parent u to v
        else:
            self.sets[v_prnt].extend(self.sets[u_prnt])
            self.sets.pop(u_prnt)
    
    
    
    # find the parent of u and v vertices and return the parents
    def find_parent(self, u, v, items):
        u_key = v_key = 0
        for item in list(items):
            # item[0] is the key in dictionary
            # item[1] is the values in the dictionary
            key, value = item[0], item[1] 
            #if_true = all(x in value for x in[u, v]) # check if both u and v are in the same set
            # check the vertices in the value list and return the key as the parent of the vertex
            if u in value:
                u_key = item[0]
            if v in value:
                v_key = item[0]
            if u_key and v_key:
                break
        return (u_key, v_key)
    
    
    def connecting(self, mst, all_edges):
        leng = len(mst)
        leng -= 1
        first_node = mst[0][0]
        last_node = mst[leng][1]
        for e in all_edges:
            u,v,w = e
            if (first_node == u and last_node == v):
                mst.append(e)
            if (first_node == v and last_node == u):
                mst.append(e) 
        return mst
    
    
    
    # make the MST tree
    def execute(self):
#         mst_weight = 0
#         lenv = len(self.graph.V) # number of vertices
        
        # sorting the edges based on the wight of the edges    
        E = sorted(self.graph.E, key = lambda m: m[2])
#         print("this: ", E)
        
        self.make_sets() # make a set of vertices
        # make a list of sets of key and value pairs to iterate through them
        items = self.sets.items()
        for e in E:
            # check if number of edges in MST are less than  nodes are 
            if((len(self.MST)+1) <  int(self.graph.num_E)):
                u, v, w = e
                u_parent, v_parent = self.find_parent(u, v, items)
                # if the vertices(u,v) are not in the same sets
                if (u_parent != v_parent):
                    self.union(u_parent, v_parent)
                    # add the edge to the MST[]
                    self.MST.append(e)
            # if the MST is completed, stop looping through the edges
            else:
                break
                
        self.MST = self.connecting(self.MST, E)
        
        return self.MST

    
    # calculate the final weight of the MST
    def MSTweight_EK(self):
        sum = 0
        for (u ,v, w) in self.MST:
            sum = sum + w
        return sum
        
    
    def DFS(self, s, mst_adjmatrix):
        visited = []
#         wtt = []
#         print(s)
#         print(mst_adjmatrix[s])
#         weight_graph = 0
#         wtt = self.DFS_search(s, wtt, visited, mst_adjmatrix)
        self.DFS_search(s, visited, mst_adjmatrix)
#         print(f"\nvisited: {visited}")
#         print(f"\nlen: {len(visited)}")
        ls = [int(x) for x in visited]
#         print(f"\nvisited: {sorted(ls)}")
#         print(f"\nlen: {len(ls)}")
        
#         print(f"tsp w: {weight_graph}")
#         return wtt
        return visited
        
    
    
        
#     def DFS_search(self, s, wt, visited, mst_adjmatrix):
    def DFS_search(self, s, visited, mst_adjmatrix):
        visited.append(s)
        print(f"{s}->", end="")
        
        for children in mst_adjmatrix[s]:
#             print(f"children: {children}")
#             print(f"children: {children[0]}")
#             print(f"children weight: {children[1]}")
            if children[0] not in visited:
#                 wt.append(children[1])
#                 print(wt)
#                 self.DFS_search(children[0],wt, visited, mst_adjmatrix)
                self.DFS_search(children[0], visited, mst_adjmatrix)
#         return wt
    
    
    def find_edge_weight(self, dfs_u, dfs_v, all_edges):
        for e in all_edges:
            u,v,w = e
            if (dfs_u == u and dfs_v == v):
                return w
            if (dfs_u == v and dfs_v == u):
                return w
        
    
    def dfs_weights(self, dfs_list, all_edges):
#         print("\ndef_list: ", dfs_list)
        total_weight = 0
        print("\n-----------------------")
        for i in range(len(dfs_list)):
            if (i+1) == len(dfs_list):
                break
            dfs_u = dfs_list[i]
            dfs_v = dfs_list[i+1]
            print(f"dfs_list[{i}]: {dfs_list[i]}, dfs_list[{i+1}]: {dfs_list[i+1]}")
            total_weight += self.find_edge_weight(dfs_u, dfs_v, all_edges)
        return total_weight

In [72]:
twoAppSol = []
dfs_total_weight = 0
for file in os.listdir():
    NAME , EDGE_WEIGHT_TYPE , DIMENSION , Coordinates = parse(file)
    vertices, edges = makeGraph(Coordinates)
#     print(edges)
    num_V_E = [len(vertices), len(edges)]
    gc.disable()
    start_time = perf_counter_ns()
    graph = Graph(vertices, edges, num_V_E[0], num_V_E[1])
    algo = Kruskal_Efficient(graph)
    result = algo.execute()
#     ww = 0
    adj_martix = make_set_nn(result)
#     print("this is adj_list: ", adj_mrtx)
    s_node = result[0][0]
    dfs_list = algo.DFS(s_node, adj_martix)
    dfs_total_weight = algo.dfs_weights(dfs_list, edges)
    print(f"dfs_total_weight: {dfs_total_weight}")
#     print("\nlist: ", ww)

#     print("wtt", ww)
#     f_w = 0
#     for w in ww:
#         f_w += w
#     print(f"fw: {f_w}")
    
    weight = algo.MSTweight_EK()
    end_time = perf_counter_ns()
    time = end_time - start_time
    gc.enable()
    print("\n\n")
    print("did",NAME,"in",time,"ns with a weight of:",weight)
    print("----------------------------------------------------")
    twoAppSol.append((NAME,weight,time))

35->36->39->40->38->24->48->46->5->15->43->33->6->4->25->12->28->27->26->47->13->14->52->51->11->37->49->32->45->19->41->8->10->9->1->22->31->18->3->17->21->42->7->2->34->44->16->50->20->23->30->29->
-----------------------
dfs_list[0]: 35, dfs_list[1]: 36
dfs_list[1]: 36, dfs_list[2]: 39
dfs_list[2]: 39, dfs_list[3]: 40
dfs_list[3]: 40, dfs_list[4]: 38
dfs_list[4]: 38, dfs_list[5]: 24
dfs_list[5]: 24, dfs_list[6]: 48
dfs_list[6]: 48, dfs_list[7]: 46
dfs_list[7]: 46, dfs_list[8]: 5
dfs_list[8]: 5, dfs_list[9]: 15
dfs_list[9]: 15, dfs_list[10]: 43
dfs_list[10]: 43, dfs_list[11]: 33
dfs_list[11]: 33, dfs_list[12]: 6
dfs_list[12]: 6, dfs_list[13]: 4
dfs_list[13]: 4, dfs_list[14]: 25
dfs_list[14]: 25, dfs_list[15]: 12
dfs_list[15]: 12, dfs_list[16]: 28
dfs_list[16]: 28, dfs_list[17]: 27
dfs_list[17]: 27, dfs_list[18]: 26
dfs_list[18]: 26, dfs_list[19]: 47
dfs_list[19]: 47, dfs_list[20]: 13
dfs_list[20]: 13, dfs_list[21]: 14
dfs_list[21]: 14, dfs_list[22]: 52
dfs_list[22]: 52, dfs_list[23]:

193->195->199->205->188->192->191->187->168->164->153->141->124->118->111->104->103->109->200->204->207->221->220->219->213->217->212->208->206->211->223->231->236->230->222->215->198->197->185->179->166->167->162->160->156->147->148->150->144->140->138->130->134->133->131->135->137->136->129->132->125->126->121->122->128->139->116->87->86->85->84->63->62->61->83->80->81->70->68->69->67->66->78->74->73->72->71->82->79->96->77->76->75->64->60->59->58->55->54->56->57->53->14->13->15->16->17->18->20->19->12->11->10->9->7->8->6->5->4->2->3->1->65->127->123->120->115->114->110->102->95->100->105->108->112->113->119->117->107->101->98->97->99->106->88->89->90->93->92->91->94->142->146->159->154->152->145->189->186->194->180->174->176->175->181->173->171->170->163->161->155->149->190->201->210->225->224->235->266->269->301->22->23->24->33->34->35->36->38->37->39->40->41->42->45->47->46->43->44->48->49->50->52->51->31->30->29->28->27->32->21->216->209->218->227->233->238->245->244->243->256->2

dfs_list[226]: 261, dfs_list[227]: 262
dfs_list[227]: 262, dfs_list[228]: 263
dfs_list[228]: 263, dfs_list[229]: 264
dfs_list[229]: 264, dfs_list[230]: 279
dfs_list[230]: 279, dfs_list[231]: 280
dfs_list[231]: 280, dfs_list[232]: 281
dfs_list[232]: 281, dfs_list[233]: 282
dfs_list[233]: 282, dfs_list[234]: 283
dfs_list[234]: 283, dfs_list[235]: 284
dfs_list[235]: 284, dfs_list[236]: 285
dfs_list[236]: 285, dfs_list[237]: 286
dfs_list[237]: 286, dfs_list[238]: 287
dfs_list[238]: 287, dfs_list[239]: 290
dfs_list[239]: 290, dfs_list[240]: 291
dfs_list[240]: 291, dfs_list[241]: 292
dfs_list[241]: 292, dfs_list[242]: 293
dfs_list[242]: 293, dfs_list[243]: 294
dfs_list[243]: 294, dfs_list[244]: 295
dfs_list[244]: 295, dfs_list[245]: 296
dfs_list[245]: 296, dfs_list[246]: 312
dfs_list[246]: 312, dfs_list[247]: 313
dfs_list[247]: 313, dfs_list[248]: 309
dfs_list[248]: 309, dfs_list[249]: 314
dfs_list[249]: 314, dfs_list[250]: 327
dfs_list[250]: 327, dfs_list[251]: 338
dfs_list[251]: 338, dfs_l

dfs_list[447]: 305, dfs_list[448]: 240
dfs_list[448]: 240, dfs_list[449]: 342
dfs_list[449]: 342, dfs_list[450]: 334
dfs_list[450]: 334, dfs_list[451]: 371
dfs_list[451]: 371, dfs_list[452]: 370
dfs_list[452]: 370, dfs_list[453]: 369
dfs_list[453]: 369, dfs_list[454]: 398
dfs_list[454]: 398, dfs_list[455]: 391
dfs_list[455]: 391, dfs_list[456]: 382
dfs_list[456]: 382, dfs_list[457]: 390
dfs_list[457]: 390, dfs_list[458]: 411
dfs_list[458]: 411, dfs_list[459]: 397
dfs_list[459]: 397, dfs_list[460]: 246
dfs_list[460]: 246, dfs_list[461]: 247
dfs_list[461]: 247, dfs_list[462]: 248
dfs_list[462]: 248, dfs_list[463]: 249
dfs_list[463]: 249, dfs_list[464]: 250
dfs_list[464]: 250, dfs_list[465]: 251
dfs_list[465]: 251, dfs_list[466]: 252
dfs_list[466]: 252, dfs_list[467]: 253
dfs_list[467]: 253, dfs_list[468]: 254
dfs_list[468]: 254, dfs_list[469]: 239
dfs_list[469]: 239, dfs_list[470]: 265
dfs_list[470]: 265, dfs_list[471]: 237
dfs_list[471]: 237, dfs_list[472]: 232
dfs_list[472]: 232, dfs_l

dfs_list[38]: 1000, dfs_list[39]: 526
dfs_list[39]: 526, dfs_list[40]: 242
dfs_list[40]: 242, dfs_list[41]: 553
dfs_list[41]: 553, dfs_list[42]: 70
dfs_list[42]: 70, dfs_list[43]: 126
dfs_list[43]: 126, dfs_list[44]: 294
dfs_list[44]: 294, dfs_list[45]: 689
dfs_list[45]: 689, dfs_list[46]: 831
dfs_list[46]: 831, dfs_list[47]: 165
dfs_list[47]: 165, dfs_list[48]: 351
dfs_list[48]: 351, dfs_list[49]: 458
dfs_list[49]: 458, dfs_list[50]: 692
dfs_list[50]: 692, dfs_list[51]: 863
dfs_list[51]: 863, dfs_list[52]: 445
dfs_list[52]: 445, dfs_list[53]: 471
dfs_list[53]: 471, dfs_list[54]: 166
dfs_list[54]: 166, dfs_list[55]: 401
dfs_list[55]: 401, dfs_list[56]: 933
dfs_list[56]: 933, dfs_list[57]: 461
dfs_list[57]: 461, dfs_list[58]: 322
dfs_list[58]: 322, dfs_list[59]: 775
dfs_list[59]: 775, dfs_list[60]: 90
dfs_list[60]: 90, dfs_list[61]: 839
dfs_list[61]: 839, dfs_list[62]: 307
dfs_list[62]: 307, dfs_list[63]: 41
dfs_list[63]: 41, dfs_list[64]: 116
dfs_list[64]: 116, dfs_list[65]: 418
dfs_li

...

dfs_list[902]: 367, dfs_list[903]: 954
dfs_list[903]: 954, dfs_list[904]: 901
dfs_list[904]: 901, dfs_list[905]: 993
dfs_list[905]: 993, dfs_list[906]: 444
dfs_list[906]: 444, dfs_list[907]: 272
dfs_list[907]: 272, dfs_list[908]: 735
dfs_list[908]: 735, dfs_list[909]: 941
dfs_list[909]: 941, dfs_list[910]: 808
dfs_list[910]: 808, dfs_list[911]: 233
dfs_list[911]: 233, dfs_list[912]: 187
dfs_list[912]: 187, dfs_list[913]: 754
dfs_list[913]: 754, dfs_list[914]: 782
dfs_list[914]: 782, dfs_list[915]: 315
dfs_list[915]: 315, dfs_list[916]: 805
dfs_list[916]: 805, dfs_list[917]: 88
dfs_list[917]: 88, dfs_list[918]: 323
dfs_list[918]: 323, dfs_list[919]: 474
dfs_list[919]: 474, dfs_list[920]: 343
dfs_list[920]: 343, dfs_list[921]: 261
dfs_list[921]: 261, dfs_list[922]: 303
dfs_list[922]: 303, dfs_list[923]: 705
dfs_list[923]: 705, dfs_list[924]: 89
dfs_list[924]: 89, dfs_list[925]: 922
dfs_list[925]: 922, dfs_list[926]: 258
dfs_list[926]: 258, dfs_list[927]: 572
dfs_list[927]: 572, dfs_list[

did gr202 in 358782800 ns with a weight of: 48898.41192029638
----------------------------------------------------
110->124->114->38->111->125->126->166->127->32->128->129->130->138->139->143->140->141->144->145->146->147->148->151->149->205->154->155->150->153->152->142->131->132->171->169->167->39->40->170->133->134->135->136->137->168->179->177->174->172->163->181->180->178->182->183->185->186->187->197->198->196->195->194->192->193->191->190->189->188->49->45->44->43->199->184->46->42->50->48->47->175->176->173->164->165->162->161->160->159->156->157->158->204->212->206->207->208->209->210->201->203->202->221->222->223->220->219->217->218->225->224->227->226->228->229->216->215->214->213->211->200->41->29->117->109->108->113->116->119->122->121->123->120->118->104->105->107->97->98->96->106->112->115->99->95->37->35->36->101->102->103->100->94->93->92->91->75->74->90->89->86->85->25->22->21->15->14->13->2->26->27->28->87->80->88->79->78->70->71->69->68->67->72->73->84->24->23->82->

15->17->11->32->91->98->23->60->62->86->27->12->20->57->7->9->87->51->61->25->81->69->73->50->44->2->68->85->39->30->96->78->52->5->37->33->76->13->95->82->64->40->54->58->28->93->67->55->83->34->29->46->3->43->14->71->41->100->48->35->77->45->47->74->21->72->10->84->90->49->6->63->1->92->8->42->89->31->80->56->97->75->19->53->79->18->24->38->36->99->88->16->22->94->70->4->65->66->26->59->
-----------------------
dfs_list[0]: 15, dfs_list[1]: 17
dfs_list[1]: 17, dfs_list[2]: 11
dfs_list[2]: 11, dfs_list[3]: 32
dfs_list[3]: 32, dfs_list[4]: 91
dfs_list[4]: 91, dfs_list[5]: 98
dfs_list[5]: 98, dfs_list[6]: 23
dfs_list[6]: 23, dfs_list[7]: 60
dfs_list[7]: 60, dfs_list[8]: 62
dfs_list[8]: 62, dfs_list[9]: 86
dfs_list[9]: 86, dfs_list[10]: 27
dfs_list[10]: 27, dfs_list[11]: 12
dfs_list[11]: 12, dfs_list[12]: 20
dfs_list[12]: 20, dfs_list[13]: 57
dfs_list[13]: 57, dfs_list[14]: 7
dfs_list[14]: 7, dfs_list[15]: 9
dfs_list[15]: 9, dfs_list[16]: 87
dfs_list[16]: 87, dfs_list[17]: 51
dfs_list[17

dfs_list[54]: 273, dfs_list[55]: 276
dfs_list[55]: 276, dfs_list[56]: 279
dfs_list[56]: 279, dfs_list[57]: 281
dfs_list[57]: 281, dfs_list[58]: 282
dfs_list[58]: 282, dfs_list[59]: 428
dfs_list[59]: 428, dfs_list[60]: 342
dfs_list[60]: 342, dfs_list[61]: 341
dfs_list[61]: 341, dfs_list[62]: 270
dfs_list[62]: 270, dfs_list[63]: 271
dfs_list[63]: 271, dfs_list[64]: 267
dfs_list[64]: 267, dfs_list[65]: 240
dfs_list[65]: 240, dfs_list[66]: 235
dfs_list[66]: 235, dfs_list[67]: 228
dfs_list[67]: 228, dfs_list[68]: 406
dfs_list[68]: 406, dfs_list[69]: 401
dfs_list[69]: 401, dfs_list[70]: 407
dfs_list[70]: 407, dfs_list[71]: 274
dfs_list[71]: 274, dfs_list[72]: 277
dfs_list[72]: 277, dfs_list[73]: 426
dfs_list[73]: 426, dfs_list[74]: 440
dfs_list[74]: 440, dfs_list[75]: 280
dfs_list[75]: 280, dfs_list[76]: 283
dfs_list[76]: 283, dfs_list[77]: 284
dfs_list[77]: 284, dfs_list[78]: 285
dfs_list[78]: 285, dfs_list[79]: 286
dfs_list[79]: 286, dfs_list[80]: 287
dfs_list[80]: 287, dfs_list[81]: 288
d

dfs_list[317]: 388, dfs_list[318]: 390
dfs_list[318]: 390, dfs_list[319]: 385
dfs_list[319]: 385, dfs_list[320]: 121
dfs_list[320]: 121, dfs_list[321]: 122
dfs_list[321]: 122, dfs_list[322]: 110
dfs_list[322]: 110, dfs_list[323]: 132
dfs_list[323]: 132, dfs_list[324]: 145
dfs_list[324]: 145, dfs_list[325]: 144
dfs_list[325]: 144, dfs_list[326]: 391
dfs_list[326]: 391, dfs_list[327]: 157
dfs_list[327]: 157, dfs_list[328]: 168
dfs_list[328]: 168, dfs_list[329]: 181
dfs_list[329]: 181, dfs_list[330]: 194
dfs_list[330]: 194, dfs_list[331]: 205
dfs_list[331]: 205, dfs_list[332]: 206
dfs_list[332]: 206, dfs_list[333]: 207
dfs_list[333]: 207, dfs_list[334]: 195
dfs_list[334]: 195, dfs_list[335]: 196
dfs_list[335]: 196, dfs_list[336]: 197
dfs_list[336]: 197, dfs_list[337]: 182
dfs_list[337]: 182, dfs_list[338]: 169
dfs_list[338]: 169, dfs_list[339]: 158
dfs_list[339]: 158, dfs_list[340]: 146
dfs_list[340]: 146, dfs_list[341]: 133
dfs_list[341]: 133, dfs_list[342]: 123
dfs_list[342]: 123, dfs_l

# Random Insertion

In [46]:
def findClosest(seen, alledges):
    sWeight = 999999
    for (u,v,w) in alledges:
        for place in seen:
            if u == place or v == place:
                if sWeight > w:
                    sWeight = w
                    smallest = (u,v,w)
    if smallest[0] == place:
        return smallest[1], smallest
    return smallest[0], smallest


def chooseRandom(unseen):
    randPlace = choice(unseen)
    return randPlace

def weightCheck(rand, seen, edges, unseen):
    bestWeight = float("inf")
    ijk = []
    ik = None
    kj = None
    ij = None
    for old1 in seen:
        cpseen = seen.copy()
        cpseen.remove(old1)
        for old2 in cpseen:
                if (old1,rand) in edges:
                    ik = edges[(old1,rand)]
                elif (rand,old1) in edges:
                    ik = edges[(rand,old1)]
                    
                if (old2,rand) in edges:
                    kj = edges[(old2,rand)]
                elif (rand,old2) in edges:
                    kj = edges[(rand,old2)]
                    
                    
                if (old1,old2) in edges:
                    ij = edges[(old1,old2)]
                elif (old2,old1) in edges:
                    ij = edges[(old2,old1)]
                    
                weight = ik[2] + kj[2] - ij[2]
                if bestWeight > weight:
                    currentBest = [ik,kj,ij]
                ik = None
                kj = None
                ij = None
    return currentBest[2],currentBest[0],currentBest[1]

def totalWeight(solution):
    return sum([edge[2] for edge in solution])

def addTheLastOne(solu, rand):
    if ('1',rand) in edgeDict:
        toadd = edgeDict[('1',rand)]
    elif (rand,'1') in edgeDict:
        toadd = edgeDict[(rand,'1')]
    return toadd

In [47]:
RanInsSol = []
for file in os.listdir():
    NAME , EDGE_WEIGHT_TYPE , DIMENSION , Coordinates = parse(file)
    edgeDict = makeDict(Coordinates)
    vertices, edges = makeGraph(Coordinates)
    num_V_E = [len(vertices), len(edges)]
    gc.disable()
    start_time = perf_counter_ns()
    solution = []
    seenPlaces = ['1'] #our starting node
    closestPlace, newEdge = findClosest(seenPlaces,edges)
    seenPlaces.append(closestPlace)
    solution.append(newEdge)
    x = seenPlaces
    y = vertices
    unseen = list(set(y) - set(x))
    while unseen:    
        randomPlace = chooseRandom(unseen)
        toRemove, toAdd1, toAdd2 = weightCheck(randomPlace, seenPlaces, edgeDict, unseen)
        solution.remove(toRemove)
        solution.append(toAdd1)
        solution.append(toAdd2)
        seenPlaces.append(randomPlace)
        x = seenPlaces
        y = vertices
        unseen = list(set(y) - set(x))
    toAdd = addTheLastOne(solution,randomPlace)
    solution.append(toAdd)
    end_time = perf_counter_ns()
    gc.enable()
    time = end_time - start_time
    weight = totalWeight(solution)
    RanInsSol.append((NAME,weight,time))
    print("did",NAME,"in",time,"ns with a weight of:",weight)

did berlin52 in 141605400 ns with a weight of: 31860.58679471314
did burma14 in 1133200 ns with a weight of: 3445.5447236645805
did ch150 in 2565752300 ns with a weight of: 54353.17934383836
did d493 in 109743108800 ns with a weight of: 467006.82124175347
did dsj1000 in 972268885500 ns with a weight of: 560597697.8682128
did eil51 in 101034900 ns with a weight of: 1563.6086728579319
did gr202 in 5661437800 ns with a weight of: 356705.3451642531
did gr229 in 8479796200 ns with a weight of: 1297735.6272894489
did kroA100 in 651748800 ns with a weight of: 162340.4388123375
did kroD100 in 617225800 ns with a weight of: 180857.46167054516
did pcb442 in 71877255400 ns with a weight of: 768162.3450971601
did ulysses16.tsp in 2611000 ns with a weight of: 14811.80800277316
did ulysses22.tsp in 7216000 ns with a weight of: 22214.862535377215


In [23]:
RanInsSol

[('berlin52', 29428.111786596794, 243054800),
 ('burma14', 2737.539853144332, 1251700),
 ('ch150', 55724.05461379605, 2312867600),
 ('d493', 443429.92589803785, 103192683600),
 ('dsj1000', 539510813.8346387, 893661997100),
 ('eil51', 1603.282360850528, 99852500),
 ('gr202', 342704.35531254986, 5729098200),
 ('gr229', 1358170.1554814645, 8402847000),
 ('kroA100', 158240.5919376652, 627420500),
 ('kroD100', 167326.36840013287, 564244600),
 ('pcb442', 757709.9369018618, 62435998600),
 ('ulysses16.tsp', 15499.728768686433, 3449400),
 ('ulysses22.tsp', 19474.773648798273, 4815300)]